#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [360]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np
from statistics import mode

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [361]:
def entropy_func(class_count, num_samples):
    p = class_count / num_samples
    return -1 * (p * np.log(p))


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size

    def group_entropy(self):
        entropy = 0
        for group in set(self.group_classes):
            entropy += entropy_func(dict(zip(*np.unique(self.group_classes, return_counts=True))).get(group), len(self.group_classes))
        return entropy

class Node:
    def __init__(self, split_feature, split_val, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data):
        if self.val is not None:
            return self.val
        return self.child_node_a.predict(data) if data[self.split_feature] < self.split_val else self.child_node_b.predict(data)


class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a:Group, group_b:Group):
        length = len(group_a.group_classes) + len(group_b.group_classes)
        a_ratio = len(group_a.group_classes) / length
        b_ratio = len(group_b.group_classes) / length
        return a_ratio * group_a.entropy + b_ratio * group_b.entropy

    def get_information_gain(self, parent_group:Group, child_group_a:Group, child_group_b:Group):
        return parent_group.entropy - self.get_split_entropy(child_group_a, child_group_b)

    def get_best_feature_split(self, feature_values, classes):
        unique_values = set(feature_values)
        parent_group = Group(classes)
        best_split, best_gain = None, 0
        for val in unique_values:
            # spliting the parent group into 2 groups, dividing by val
            smaller_values = [feature_values[i] for i in range(len(feature_values)) if feature_values[i] < val]
            smaller_classes = [classes[i] for i in range(len(feature_values)) if feature_values[i] < val]
            bigger_values = [feature_values[i] for i in range(len(feature_values)) if feature_values[i] >= val]
            bigger_classes = [classes[i] for i in range(len(feature_values)) if feature_values[i] >= val]
            child_group_a, child_group_b = Group(smaller_classes), Group(bigger_classes)
            gain = self.get_information_gain(parent_group, child_group_a, child_group_b)
            if gain > best_gain:
                best_split, best_gain = val, gain
        return (best_gain, best_split)

    def get_best_split(self, data, classes):
        best_split, split_value, best_gain = None, None, 0
        for feature in range(len(data[0])):
            feature_values = [row[feature] for row in data]
            gain, split = self.get_best_feature_split(feature_values, classes)
            if gain > best_gain:
                best_split, split_value, best_gain = feature, split, gain
        return (best_split, split_value)

    def build_tree(self, data, classes, depth=0):
        self.tree = self._build_tree_recursively(data, classes, depth)
        return self.tree

    def _build_tree_recursively(self, data, classes, depth):
        if len(set(classes)) == 1:
            return Node(None, None, 0, None, None, classes[0])
        if depth == 0:
            # return Node containing most frequent value in classes
            return Node(None, None, 0, None, None, mode(classes))
        best_split, split_value = self.get_best_split(data, classes)
        child_a_data = [data[i] for i in range(len(data)) if data[i][best_split] < split_value]
        child_a_classes = [classes[i] for i in range(len(data)) if data[i][best_split] < split_value]
        child_b_data = [data[i] for i in range(len(data)) if data[i][best_split] >= split_value]
        child_b_classes = [classes[i] for i in range(len(data)) if data[i][best_split] >= split_value]        
        return Node(best_split, split_value, depth, self._build_tree_recursively(child_a_data, child_a_classes, depth-1), self._build_tree_recursively(child_b_data, child_b_classes, depth-1))

    def predict(self, data):
        return self.tree.predict(data)

In [362]:
dc = DecisionTreeClassifier(3)
dc.build_tree(x_train, y_train, 3)
correct = 0
for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    #print(f"Prediction = {prediction} Actual = {gt}")
    correct += 1 if prediction == gt else 0
acc = correct / y_test.size
print(f"Acc = {acc}")

Acc = 0.9777777777777777


In [363]:
"""
Test	
1	0.6				
2	0.8667				
3	0.9333

Test size 0.15					  |  avg    
1	0,652	0,608	0,695	0,521 | 0,619
2	0,913	0,869	1	    0,913 |	0,923
3	0,913	0,956	0,826	1	  | 0,923

Test size 0.2					
1	0,43	0,633	0,7	    0,6	  | 0,590
2	0,8	    1	    0,966	0,9	  | 0,916
3	0,866	0,9	    0,93	0,866 |	0,883

Test size 0.3					
1	0,644	0,6	    0,577	0,711 | 0,633
2	0,955	0,977	0,866	0,933 |	0,932
3	0,9777	0,955	0,911	1	  | 0,961
"""

'\nTest\t\n1\t0.6\t\t\t\t\n2\t0.8667\t\t\t\t\n3\t0.9333\n\nTest size 0.15\t\t\t\t\t  |  avg    \n1\t0,652\t0,608\t0,695\t0,521 | 0,619\n2\t0,913\t0,869\t1\t    0,913 |\t0,923\n3\t0,913\t0,956\t0,826\t1\t  | 0,923\n\nTest size 0.2\t\t\t\t\t\n1\t0,43\t0,633\t0,7\t    0,6\t  | 0,590\n2\t0,8\t    1\t    0,966\t0,9\t  | 0,916\n3\t0,866\t0,9\t    0,93\t0,866 |\t0,883\n\nTest size 0.3\t\t\t\t\t\n1\t0,644\t0,6\t    0,577\t0,711 | 0,633\n2\t0,955\t0,977\t0,866\t0,933 |\t0,932\n3\t0,9777\t0,955\t0,911\t1\t  | 0,961\n'